In [17]:
import torchvision
import torch
from torchvision import datasets, transforms
import numpy as np

In [18]:
data_train = datasets.MNIST(root = "./data/",
                            transform=transforms,
                            train = True,
                            download = True)

data_test = datasets.MNIST(root="./data/",
                           transform = transforms,
                           train = False)

In [19]:
def read_label(file_name):
    file_handle = open(file_name, "rb")  # 以二进制打开文档
    file_content = file_handle.read()  # 读取到缓冲区中

    head = struct.unpack_from('>II', file_content, 0)  # 取前2个整数，返回一个元组
    offset = struct.calcsize('>II')

    labelNum = head[1]  # label数
    # print(labelNum)
    bitsString = '>' + str(labelNum) + 'B'  # fmt格式：'>47040000B'
    label = struct.unpack_from(bitsString, file_content, offset)  # 取data数据，返回一个元组
    return np.array(label)

In [20]:
def read_image(file_name):
    #先用二进制方式把文件都读进来
    file_handle=open(file_name,"rb")  #以二进制打开文档
    file_content=file_handle.read()   #读取到缓冲区中

    offset=0
    head = struct.unpack_from('>IIII', file_content, offset)  # 取前4个整数，返回一个元组
    offset += struct.calcsize('>IIII')
    imgNum = head[1]  #图片数
    rows = head[2]   #宽度
    cols = head[3]  #高度
    images=np.empty((imgNum , 784))#empty，是它所常见的数组内的所有元素均为空，没有实际意义，它是创建数组最快的方法
    image_size=rows*cols#单个图片的大小
    fmt='>' + str(image_size) + 'B'#单个图片的format

    for i in range(imgNum):
        images[i] = np.array(struct.unpack_from(fmt, file_content, offset))
        # images[i] = np.array(struct.unpack_from(fmt, file_content, offset)).reshape((rows, cols))
        offset += struct.calcsize(fmt)
    return images

In [21]:
import struct
train_image = "data/MNIST/raw/train-images-idx3-ubyte"
test_image = "data/MNIST/raw/t10k-images-idx3-ubyte"
train_label = "data/MNIST/raw/train-labels-idx1-ubyte"
test_label = "data/MNIST/raw/t10k-labels-idx1-ubyte"
train_x = read_image(train_image)  # train_dataSet
test_x = read_image(test_image)  # test_dataSet
train_y = read_label(train_label)  # train_label
test_y = read_label(test_label)  # test_label
train_x.shape

(60000, 784)

In [6]:
from sklearn.metrics import accuracy_score
def knn2(train_idx,test_idx,k):
    train_matrix=train_x[train_idx]
    test_matrix=train_x[test_idx]
    predict_label=[]
    for test_data in test_matrix:
        class_count=[0]*10
        distance1=np.tile(test_data,(train_matrix.shape[0],1))-train_matrix
        distance2=distance1**2
        distance3=distance2.sum(axis=1)
        distance4=distance3**0.5
        sortedDistIndicies=distance4.argsort()
        
        for i in range(k):
            label=train_y[sortedDistIndicies[i]]
            class_count[label]+=1
        max=0
        id=0
        for idx in range(len(class_count)):
            if class_count[idx]>=max:
                id=idx
                max=class_count[idx]
        predict_label.append(id)
    
    acc=accuracy_score(train_y[test_idx],predict_label)
    
    print(acc)

In [48]:
import pandas as pd
train_df=pd.read_csv('train.csv')
test_df=pd.read_csv('test.csv')
train_x=train_df.values[:,1:]
train_y=train_df.values[:,0]
test_x=test_df.values


In [49]:
from sklearn.neighbors import KNeighborsClassifier

def knn(train_idx,test_idx,neighbor):
    model = KNeighborsClassifier(n_neighbors=neighbor)
    model.fit(train_x[train_idx], train_y[train_idx])
    pre=model.predict(train_x[test_idx])
#     print(accuracy_score(pre,train_y[test_idx]))
    test_pre=model.predict(test_x)
    return pre,test_pre

In [50]:
from sklearn.ensemble import RandomForestClassifier

def ramdom_forest(train_idx,test_idx,num_of_trees):
    model = RandomForestClassifier(n_estimators=num_of_trees)
    model.fit(train_x[train_idx], train_y[train_idx])

    pre = model.predict(train_x[test_idx])
#     acc_rf = accuracy_score(train_y[test_idx],y_pred_rf)
    test_pre=model.predict(test_x)
    return pre,test_pre

In [51]:
import torch as t
from tqdm import tqdm
from torchsummary import summary
class fc_net(t.nn.Module):
    '''
    全连接网络
    '''
    def __init__(self):
        super(fc_net,self).__init__()
        self.fc1 = t.nn.Sequential(t.nn.Linear(784,200),t.nn.ReLU())
        self.fc2 = t.nn.Sequential(t.nn.Linear(200,100),t.nn.ReLU())
        self.fc3 = t.nn.Sequential(t.nn.Linear(100,20),t.nn.ReLU())
        self.fc4 = t.nn.Linear(20,10)
    def forward(self,x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        return x
    
epochs=5
BATCH_SIZE=10
def conv_net(train_set,test_set):
    net = fc_net()
    criterion = t.nn.CrossEntropyLoss()
    optim = t.optim.Adam(net.parameters(),lr=0.001,weight_decay=0.0)
    
    for epoch in range(epochs):
        index=0
        train_loss = 0   
        train_acc = 0 
        for i in tqdm(range(int(len(train_set)/BATCH_SIZE)),total=int(len(train_set)/BATCH_SIZE)):
            x=train_x[train_set][index:index+BATCH_SIZE]
            y=train_y[train_set][index:index+BATCH_SIZE]
            
            x=t.tensor(x,dtype=torch.float32)
            y=t.tensor(y).long()
            
            out=net.forward(x)
            loss=criterion(out,y)
            optim.zero_grad()
            loss.backward()
            optim.step()
            index+=BATCH_SIZE
            _,pred = out.max(1)
            num_correct = (pred == y).sum()
            acc = int(num_correct) / y.shape[0]
            train_acc+=acc
            
#         print('acc', train_acc/len(train_set)*BATCH_SIZE)
    
    prediction=net(t.tensor(train_x[test_set],dtype=torch.float32))
    _,pos=prediction.max(1)
    num_correct = (t.tensor(train_y[test_set]).long() == pos).sum()
    acc = int(num_correct) / len(test_set)
    print(acc)
    prediction_test_x=net(t.tensor(test_x,dtype=torch.float32))
    _,pos_test_x=prediction_test_x.max(1)
    return pos.tolist(),pos_test_x.tolist()
        
             


In [85]:
import numpy as np
k_fold=5
index_matrix=np.array([i for i in range(train_x.shape[0])])
data_length=index_matrix.shape[0]

np.random.shuffle(index_matrix)

model_1_train_data=[]
model_1_test_data=[]

model_2_train_data=[]
model_2_test_data=[]

model_3_train_data=[]
model_3_test_data=[]
num_of_each_fold=int(data_length/k_fold)
for i in range(0,data_length,num_of_each_fold):
    test_set=index_matrix[i:i+num_of_each_fold]
    train_set=list(set(index_matrix)-set(test_set))
    
    train_pre_1,test_pre_1=knn(train_set,test_set,3)
    model_1_train_data+=(list(train_pre_1))
    model_1_test_data.append(list(test_pre_1))
    
    train_pre_2,test_pre_2=ramdom_forest(train_set,test_set,3)
    model_2_train_data+=(list(train_pre_2))
    model_2_test_data.append(list(test_pre_2))
    
    train_pre_3,test_pre_3=conv_net(train_set,test_set)
    model_3_train_data+=(list(train_pre_3))
    model_3_test_data.append(list(test_pre_3))

100%|██████████| 3360/3360 [09:55<00:00,  5.64it/s]


0.9542857142857143


100%|██████████| 3360/3360 [10:49<00:00,  5.18it/s]


0.9485714285714286


100%|██████████| 3360/3360 [10:01<00:00,  5.59it/s]


0.9525


100%|██████████| 3360/3360 [10:35<00:00,  5.28it/s]


0.9583333333333334


100%|██████████| 3360/3360 [10:36<00:00,  5.28it/s]


0.9573809523809523


In [86]:
from collections import Counter
def majority_voting(data):
    l=[]
    data=np.array(data).T
    for idx in range(data.shape[0]):
        result=Counter(data[idx]).most_common(1)[0][0]
        
        l.append(result)
    return l


In [87]:
layer_1_train_x_output=np.c_[model_1_train_data,model_2_train_data,model_2_train_data]
layer_1_train_y_output=train_y[index_matrix]
layer_1_test_x_output=np.c_[majority_voting(model_1_test_data),majority_voting(model_2_test_data),majority_voting(model_3_test_data)]
# layer_1_test_y_output=test_y


In [88]:
from sklearn.linear_model import LogisticRegression
def logistice_regression(train_set,test_set):
    lr = LogisticRegression(multi_class='multinomial',solver="newton-cg")
    lr.fit(layer_1_train_x_output[train_set], layer_1_train_y_output[train_set])
    pred_train_x=lr.predict(layer_1_train_x_output[test_set])
    pred_test_x=lr.predict(layer_1_test_x_output)
    return pred_train_x,pred_test_x
#     acc=accuracy_score(pred,layer_1_train_y_output[test_set])
#     print(acc)

In [89]:
from sklearn import svm
def svm_model(train_set,test_set):
    model = svm.SVC(gamma='scale', C=4.0, decision_function_shape='ovr', kernel='rbf')
    model.fit(layer_1_train_x_output[train_set], layer_1_train_y_output[train_set])
    pred_train_x = model.predict(layer_1_train_x_output[test_set])
    pred_test_x=model.predict(layer_1_test_x_output)
    return pred_train_x,pred_test_x
#     acc=accuracy_score(pred,layer_1_train_y_output[test_set])
#     print(acc)

In [90]:
index_matrix_2=np.array([i for i in range(layer_1_train_x_output.shape[0])])
num_of_each_fold_2=int(index_matrix_2.shape[0]/5)

layer_2_model_1_train_data=[]
layer_2_model_1_test_data=[]

layer_2_model_2_train_data=[]
layer_2_model_2_test_data=[]

layer_2_model_3_train_data=[]
layer_2_model_3_test_data=[]
for i in range(0,index_matrix_2.shape[0],num_of_each_fold_2):
    test_set_2=index_matrix_2[i:i+num_of_each_fold_2]
    train_set_2=list(set(index_matrix_2)-set(test_set_2))
    
    x,y=logistice_regression(train_set_2,test_set_2)
    layer_2_model_1_train_data+=list(x)
    layer_2_model_1_test_data.append(list(y))
    
    x1,y1=svm_model(train_set_2,test_set_2)
    layer_2_model_2_train_data+=list(x1)
    layer_2_model_2_test_data.append(list(y1))
    
    x2,y2=knn(train_set_2,test_set_2,3)
    layer_2_model_3_train_data+=list(x2)
    layer_2_model_3_test_data.append(list(y2))
    

In [91]:
layer_2_train_x_output=np.c_[layer_2_model_1_train_data,layer_2_model_2_train_data,layer_2_model_3_train_data]
layer_2_train_y_output=train_y[index_matrix_2]
layer_2_test_x_output=np.c_[majority_voting(layer_2_model_1_test_data),majority_voting(layer_2_model_2_test_data),majority_voting(layer_2_model_3_test_data)]
# layer_2_test_y_output=test_y

In [118]:
r=[list(np.unique(data)) for data in layer_2_test_x_output if np.unique(data).shape[0]!=1]
# [i.shape[0] for i in r ]  
1-np.array(r).shape[0]/len(test_x)
# np.array(r).shape


0.9741785714285714

In [98]:
def final_model():
    model = KNeighborsClassifier(n_neighbors=2)
    model.fit(layer_2_train_x_output, layer_2_train_y_output)
    pre=model.predict(layer_2_test_x_output)
    return pre

In [133]:
from sklearn import linear_model
from sklearn.metrics import accuracy_score
def linear_regression(train_x,train_y):
    
    reg = linear_model.LinearRegression()
    reg.fit(train_x,train_y)
    pred=reg.predict(layer_2_test_x_output)
    return pred

In [137]:
final_pred=final_model()
# result=np.around(final_pred)
# final_pred=majority_voting(layer_2_test_x_output.T)


In [138]:
submit_df=pd.read_csv('sample_submission.csv')
newDf = pd.DataFrame(submit_df, columns=['ImageId'])
newDf['Label']=final_pred
newDf.to_csv('stacking_net.csv',index=False)

In [139]:
pd.read_csv('stacking_net.csv')

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9
